In [1]:
import numpy as np
import random
from torch.utils.data import DataLoader
from torchvision import transforms
import torch
import time
from time import gmtime, strftime
import os
from data.data_split import cyclic_split
from data.dataset import get_dataset_class, CustomTensorDataset_GBA, CustomTensorDataset_GBA_gap
from data.transforms import ClassifyByThresholds
from trainer import NIMSTrainer_Germnay_Two
from model.swinunet_model import SwinUnet_CAM_Two, SwinUnet_Two
from model.conv_lstm import ConvLSTM,ConvLSTM_Two
from losses import *
from utils import *
import torch.optim as optim
import warnings
import sys
import datetime
with warnings.catch_warnings():
    warnings.simplefilter("ignore", category=RuntimeWarning)

import wandb

In [2]:
def seed_worker(worker_id):
    worker_seed = torch.initial_seed() % 2**32
    np.random.seed(worker_seed)
    random.seed(worker_seed)

def main(args, wandb):
    device = set_device(args)
    fix_seed(args.seed)
    g = torch.Generator()
    g.manual_seed(args.seed)
    

    # Set experiment name and use it as process name if possible
    experiment_name = get_experiment_name(args)
    current_time = strftime("%Y-%m-%d %H:%M:%S", time.localtime(time.time()))
    args.experiment_name = experiment_name+ "_" + current_time
    experiment_name = args.experiment_name
    
    # print('Running Experiment'.center(30).center(80, "="))
    # print(experiment_name)

    save_path = '/home/jianer/github_proj/PostRainBenchTest-main/3_GBA_wandb_ConvLSTM/GBA_dataset/experiment/'
    trn_x_1 = np.load(save_path + 'X_train_period1.npy')
    trn_x_2= np.load(save_path + 'X_train_period2.npy')
    trn_y_1 = np.load(save_path + 'y_train_period1.npy')
    trn_y_2 = np.load(save_path + 'y_train_period2.npy')

    tst_x = np.load(save_path + 'X_test_period.npy')
    tst_y = np.load(save_path + 'y_test_period.npy')
    vld_x = np.load(save_path + 'X_valid_period.npy')
    vld_y = np.load(save_path + 'y_valid_period.npy')

    print('Load datasets in CPU memory successfully!')
    print("#" * 80)

    batch_size = args.batch_size
    train_dataset = CustomTensorDataset_GBA_gap(torch.from_numpy(trn_x_1),torch.from_numpy(trn_x_2), \
                                                    torch.from_numpy(trn_y_1), torch.from_numpy(trn_y_2), \
                                                    args.rain_thresholds, downscaling_t=4)
    val_dataset = CustomTensorDataset_GBA(torch.from_numpy(vld_x),torch.from_numpy(vld_y), args.rain_thresholds, downscaling_t=4)
    test_dataset = CustomTensorDataset_GBA(torch.from_numpy(tst_x),torch.from_numpy(tst_y), args.rain_thresholds, downscaling_t=4)
    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, drop_last=True)
    valid_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)
    test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)
    # for x, y, z in train_dataset:
    #     print(f'x shape: {x.shape}')
    #     print(f'y shape: {y.shape}')
    #     print(f'z shape: {z.shape}')
    #     break  # 打印一次后跳出循环
    # for x, y, z in train_loader:
    #     print(f'x shape: {x.shape}')
    #     print(f'y shape: {y.shape}')
    #     print(f'z shape: {z.shape}')
    #     break  # 打印一次后跳出循环
    nwp_sample = torch.rand(1, 1, 84, 64, 64)
    # set model
    model = SwinUnet_Two(img_size=64, in_chans=84, num_classes=len(args.rain_thresholds)+1)
    
    criterion = CrossEntropyLoss_Two(args=args,
                                    device=device,
                                    num_classes=args.num_classes,
                                    experiment_name=experiment_name)
    # if finetune:
    #     checkpoint = torch.load(model_path)
    #     model.load_state_dict(checkpoint, strict=False)
        
    dice_criterion = None
    normalization = None
    
    if args.optimizer == 'sgd':
        optimizer = optim.SGD(model.parameters(), lr=args.lr, momentum=args.momentum,
                              weight_decay=args.wd, nesterov=args.nesterov)
    elif args.optimizer == 'adam':
        optimizer = optim.Adam(model.parameters(), lr=args.lr, weight_decay=args.wd)
    elif args.optimizer == 'rmsprop':
        optimizer = optim.RMSprop(model.parameters(), lr=args.lr,
                                  alpha=0.9, eps=1e-6)
    elif args.optimizer == 'adadelta':
        optimizer = optim.Adadelta(model.parameters(), lr=args.lr)

    # scheduler = optim.lr_scheduler.StepLR(optimizer, args.wd_ep)
    scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', factor=wandb.config['lr_decay_rate'], patience=10, threshold=0.0001)
    
    nims_trainer = NIMSTrainer_Germnay_Two(wandb, model, criterion, dice_criterion, optimizer, scheduler, device,
                                train_loader, valid_loader, test_loader, experiment_name,
                                args, normalization=normalization)
    # Train model
    nims_trainer.train()

In [3]:
sweep_config = {
    'method': 'random'
    }

metric = {
    'name': 'loss',
    'goal': 'minimize'   
    }

sweep_config['metric'] = metric

sweep_config['parameters'] = {}

# 常量型超参数
sweep_config['parameters'].update({
    'seed': {'value': 11611801},      
    'n_epochs': {'value': 1000},
})
    
# 离散型超参数
sweep_config['parameters'].update({
    'train_batch_size': {
        'values': [8, 16, 45, 120, 240, 384, 600]
    },
    'early_stop': {
        'values': [50]
    },
    'tolerate_loss':{
        'values':[1e-3]
    },
    'CELWeight': {
        'values': [2,4,5,6]
    },
})

    
# 连续型超参数
sweep_config['parameters'].update({
    'learning_rate': {
        'distribution': 'uniform',
        'min': 1e-6,
        'max': 1e-1
      },
    'alpha': {
        'distribution': 'uniform',
        'min': 1e-1,
        'max': 1e2,
      },
    'lr_decay_rate': {
        'distribution': 'uniform',
        'min': 1e-1,
        'max': 8e-1,
    },
})

In [4]:
def training():
     # 初始化wandb
    ############################################################################################
    nowtime = datetime.datetime.now().strftime('%Y_%m_%d_%H%M%S')
    wandb.init(
      project='PRBenchTest_Swin-Transformer_GBA_2015_2022', 
      name=nowtime, 
      )
    config = wandb.config
    ############################################################################################

    # 构建命令行参数
    sys.argv = [
    '--model', 'SwinUnet_Two',
    '--device', '0',
    '--seed', str(config['seed']),
    '--input_data', 'gdaps_kim',
    '--num_epochs', str(config['n_epochs']),
    '--rain_thresholds', '0.4', '52.0', '100.0',
    '--log_dir', 'logs/logs_1111_China',
    '--batch_size', str(config['train_batch_size']),
    '--lr', str(config['learning_rate']),
    '--use_two',
    '--loss', 'ce+mse',
    '--alpha', str(config['alpha']),
    '--kernel_size', '3',
    '--weight_version', str(config['CELWeight']),
    '--wd_ep', '100',
    '--custom_name', 'PRBenchTest_Swin-Transformer_GBA_2015_2022'
    ]
    # 模型训练
    args = parse_args(sys.argv)
    main(args, wandb)
    # best_model, best_loss = trainer(train_loader, valid_loader, model, wandb, device)
    
    # 保存模型
    # if best_loss<0.3:
    #     save_name = os.path.join(config['model_save_dir'], nowtime + '.ckpt')
    #     torch.save(best_model.state_dict(), save_name)
    #     arti_code = wandb.Artifact('ipynb', type='code')
    #     arti_code.add_file(os.path.join(config['root'], 'SURROGATE_TRAINING_WANDB.ipynb'))
    #     arti_code.add_file(os.path.join(config['root'], 'LSMDataset.py'))
    #     arti_code.add_file(os.path.join(config['root'], 'LSMLoss.py'))
    #     arti_code.add_file(os.path.join(config['root'], 'LSMTransformer.py'))
                                              
    #     # arti_model = wandb.Artifact('model', type='model')
    #     # arti_model.add_file(save_name)
    #     wandb.log_artifact(arti_code)
    #     wandb.log_artifact(arti_model)
    wandb.finish()

In [5]:
wandb.login()

wandb: Currently logged in as: panj1018 (panj1018-hong-kong-university-of-science-and-technology). Use `wandb login --relogin` to force relogin


True

In [6]:
sweep_id = wandb.sweep(sweep_config, project='PRBenchTest_Swin-Transformer_GBA_2015_2022')
print(sweep_id)

Create sweep with ID: cwebuxgw
Sweep URL: https://wandb.ai/panj1018-hong-kong-university-of-science-and-technology/PRBenchTest_Swin-Transformer_GBA_2015_2022/sweeps/cwebuxgw
cwebuxgw


In [ ]:
# wandb.agent(project='PRBenchTest_Swin-Transformer_GBA_2015_2022', sweep_id='vy7tsa9d', function=training, count=50)
wandb.agent(sweep_id, training, count=10)

wandb: Agent Starting Run: pg7a4zu8 with config:
wandb: 	CELWeight: 4
wandb: 	alpha: 81.3473942293963
wandb: 	early_stop: 50
wandb: 	learning_rate: 0.0629589129692398
wandb: 	lr_decay_rate: 0.4341205519442749
wandb: 	n_epochs: 1000
wandb: 	seed: 11611801
wandb: 	tolerate_loss: 0.001
wandb: 	train_batch_size: 8
wandb: WARNING Ignored wandb.init() arg project when running a sweep.


Global seed set to 11611801


Load datasets in CPU memory successfully!
################################################################################
SwinTransformerSys_Two expand initial----depths:[2, 2, 2, 2];depths_decoder:[1, 2, 2, 2];drop_path_rate:0.0;num_classes:4


/home/jianer/miniconda3/envs/PRBench/lib/python3.8/site-packages/torch/functional.py:478: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at  /opt/conda/conda-bld/pytorch_1659484810403/work/aten/src/ATen/native/TensorShape.cpp:2894.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]


Omitting intermediate evaluation on test set
